In [1]:
import json
from src.models.conv_model import build_model as build_conv_model
from src.data_loader.RNASeqLoader import RNASeqDataGenerator

Using TensorFlow backend.


In [2]:
json_path = "models/cDNA-ABE/search-space-0.json"
data = None
with open(json_path) as f:
    data = json.load(f)
seq_length = data['seq_length']
dim = data['dim']
filters = data['filters']
kernel_sizes = data['kernel_size']
dense_net = data['dense_net']
blocks = data['blocks']
dil_rate = data['dil_rate']
dropout = data['dropout']
pooling_size = data['pooling_size']

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

train_generator = RNASeqDataGenerator("data/processed/cDNA-ABE/train_data.hdf5", 1024)
validation_generator = RNASeqDataGenerator("data/processed/cDNA-ABE/validation_data.hdf5", 1024)

In [4]:
search_space = list()
for a in filters:
    for b in kernel_sizes:
        for c in dense_net:
            for d in blocks:
                for e in dropout:
                    for f in pooling_size:
                        for g in dil_rate:
                            search_space.append([seq_length, dim, a, b, c, d, g, e, f])

In [5]:
print("{} Models to Try.".format(len(search_space)))

24 Models to Try.


In [ ]:
from src.models.conv_model import build_model as build_conv_model
import tensorflow as tf
from time import time_ns
import os
base_path = "models/cDNA-ABE/gridsearch/"
for x in search_space:
    
    t = time_ns()
    folder_name = base_path+""+str(x)+"-"+str(t)
    os.mkdir(folder_name)
    
    model = build_conv_model(x[0], x[1], x[2], x[3], x[4], blocks=x[5], dil_rate=x[6], dropout=x[7], pooling_size=x[8])
    
    checkpoint_filepath = folder_name+"/"+str(t)+".h5"
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=False,
        monitor='val_loss',
        mode='min',
        save_best_only=True)
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    
    history = model.fit(x=train_generator, epochs=5, validation_data=validation_generator, callbacks=[model_checkpoint_callback, es], use_multiprocessing=True, workers=20)
    min_loss = min(history.history['val_loss'])
    
    json_output_file = folder_name+"/"+str(t)+".json"
    data = {}
    data['seq_length'] = x[0]
    data['dim'] = x[1]
    data['filters'] = x[2]
    data['kernel_size'] = x[3]
    data['dense_net'] = x[4]
    data['blocks'] = x[5]
    data['dil_rate'] = x[6]
    data['dropout'] = x[7]
    data['pooling_size'] = x[8]
    data.update(history.history)
    with open(json_output_file, 'w') as outfile:
        json.dump(str(data), outfile)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Epoch 1/5
2638/2638 [==============================] - 830s 314ms/step - loss: 0.0013 - mean_absolute_error: 0.0149 - mean_squared_error: 0.0013 - val_loss: 0.0013 - val_mean_absolute_error: 0.0144 - val_mean_squared_error: 0.0013
Epoch 2/5
2638/2638 [==============================] - 805s 305ms/step - loss: 0.0012 - mean_absolute_error: 0.0149 - mean_squared_error: 0.0012 - val_loss: 0.0013 - val_mean_absolute_error: 0.0165 - val_mean_squared_error: 0.0013
Epoch 3/5
2636/2638 [============================>.] - ETA: 0s - loss: 0.0011 - mean_absolute_error: 0.0148 - mean_squared_error: 0.0011Epoch 1/5
Epoch 1/5
2638/2638 [==============================] - 781s 296ms/step - loss: 0.0011 - mean_absolute_error: 0.0148 - mean_squared_error: 0.0011 - val_loss: 0.0013 - val_mean_absolute_error: 0.0146 - val_mean_squared_error: 0.0013
Epoch 4/5
2638/2638 [==============================] - 781s 296ms/step - loss: 0

Epoch 1/5
2638/2638 [==============================] - 811s 308ms/step - loss: 0.0013 - mean_absolute_error: 0.0153 - mean_squared_error: 0.0013 - val_loss: 0.0014 - val_mean_absolute_error: 0.0195 - val_mean_squared_error: 0.0014
Epoch 2/5
2638/2638 [==============================] - 796s 302ms/step - loss: 0.0012 - mean_absolute_error: 0.0150 - mean_squared_error: 0.0012 - val_loss: 0.0014 - val_mean_absolute_error: 0.0220 - val_mean_squared_error: 0.0014
Epoch 3/5
2638/2638 [==============================] - 793s 301ms/step - loss: 0.0011 - mean_absolute_error: 0.0149 - mean_squared_error: 0.0011 - val_loss: 0.0013 - val_mean_absolute_error: 0.0193 - val_mean_squared_error: 0.0013
Epoch 4/5
2638/2638 [==============================] - 797s 302ms/step - loss: 0.0011 - mean_absolute_error: 0.0148 - mean_squared_error: 0.0011 - val_loss: 0.0013 - val_mean_absolute_error: 0.0200 - val_mean_squared_error: 0.0013
Epoch 5/5
2638/2638 [==============================] - 797s 302ms/step - los